In [1]:
import pybullet as p
import pybullet_data
import pathlib
import pandas as pd
import numpy as np
import time
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from Gantry.envs.GantrySimulation import GantrySimulation
from Gantry.controller.SNS_layer import SNS_layer, SENSORY_LAYER_1_INPUT_SIZE, SENSORY_LAYER_1_SIZE, SENSORY_LAYER_2_INPUT_SIZE, SENSORY_LAYER_2_SIZE, COMMAND_LAYER_SIZE, INTER_LAYER_1_SIZE, INTER_LAYER_2_SIZE, MOTOR_LAYER_SIZE, THETA_MAX, THETA_MIN, F_MAX, F_MIN, sensory_layer_1, sensory_layer_2, R, perceptor, controller

alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!
alloc!


ImportError: cannot import name 'controller' from 'Gantry.controller.SNS_layer' (c:\Users\Yanjun\Documents\MATLAB\FRR_SoftGrasper\PyBullet Model\Gantry\controller\SNS_layer.py)

In [ ]:
perceptor._sensory_layer_1._params["tau"].data = torch.Tensor(0.0*np.ones(SENSORY_LAYER_1_SIZE))
perceptor._sensory_layer_2._params["tau"].data = torch.Tensor(0.0*np.ones(SENSORY_LAYER_2_SIZE))
perceptor._command_layer._params["tau"].data = torch.Tensor(0.0*np.ones(COMMAND_LAYER_SIZE))
controller._inter_layer_1._params["tau"].data = torch.Tensor(0.0*np.ones(INTER_LAYER_1_SIZE))
controller._inter_layer_2._params["tau"].data = torch.Tensor(0.0*np.ones(INTER_LAYER_2_SIZE))
controller._motor_layer._params["tau"].data = torch.Tensor(0.0*np.ones(MOTOR_LAYER_SIZE))

In [2]:
def pick_and_place():
    gS = GantrySimulation() #gantryURDFfile = "URDF//GrasperAndGantry//urdf//GrasperAndGantry.urdf"
    # add object to the simulation at the center of the plate
    #for row in range(2):
    #    for col in range(2):
    #        gS.addObjectsToSim("PickupCube", startPos=[-0.165 + 0.165 * col, -0.2 + 0.165 * row, (0.063+0.02)], mass_kg=1, sizeScaling=0.6,
    #                   sourceFile=str(pathlib.Path.cwd()/"Gantry\\envs\\URDF\\PickUpObject_URDF\\urdf\\PickUpObject_URDF.urdf"))

    
    gS.addObjectsToSim("PickupCube", startPos=[-0.175 * 0, 0.175 * 0, (0.063+0.02)], mass_kg=1, sizeScaling=0.6,
                       sourceFile=str(pathlib.Path.cwd()/"Gantry\\envs\\URDF\\PickUpObject_URDF\\urdf\\PickUpObject_URDF.urdf"))
    #gS.addObjectsToSim("PickupCube", startPos=[0, -0.175 * 1, (0.063+0.02)], mass_kg=1, sizeScaling=0.6,
    #                   sourceFile=str(pathlib.Path.cwd()/"Gantry\\envs\\URDF\\PickUpObject_URDF\\urdf\\PickUpObject_URDF.urdf"))
    #SoftSupportInit = p.loadURDF("URDF/SoftGrasperAssembly_SimplifiedTilt/urdf/SoftGrasperAssembly_SimplifiedTilt.urdf",
    #                              [0, 0, 0.52816* gS.lengthScale], globalScaling=gS.lengthScale, useFixedBase=False,
    #                              flags=p.URDF_USE_SELF_COLLISION_INCLUDE_PARENT | p.URDF_USE_MATERIAL_COLORS_FROM_MTL)
    #
    #
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setAdditionalSearchPath("C://Users//Ravesh//BulletPhysics//bullet3//examples//pybullet//gym//pybullet_data")

    positionset = []
    targetpositionset = []
    forceset = []
    neuronset = []

    GUI_control = True


    while (not gS.CheckStopSim()):  # check to see if the button was pressed to close the sim
        timeStart=time.perf_counter()

        GUIcontrolTarget = gS.bulletClient.readUserDebugParameter(
            gS.GUIcontrols["GUIcontrolId"])
        if GUIcontrolTarget % 2 == 0 and GUI_control is True:
            GUI_control = False
            gS.simCounter = 0
            object_position = torch.Tensor([-0.175 * 0, 0.175 * 0, -0.34]).unsqueeze(dim=0)
            target_position = torch.Tensor(
                [0.175, 0.175, -0.34]).unsqueeze(dim=0)

        ts = gS.timeStep  # time step of the simulation in seconds
        nsteps = gS.simCounter  # of simulation steps taken so far
        timev = ts*nsteps
        if timev < 1:
            target_position = torch.Tensor([-0.175 * 0, 0.175 * 0, -0.34]).unsqueeze(dim=0)
        else:
            target_position = torch.Tensor([0.175, 0.175, -0.34]).unsqueeze(dim=0)

        
        x = gS.bulletClient.getJointState(
            gS.gantryId, gS.GantryLinkIndex_dict["GantryHeadIndex"])[0] + 0.00 * np.random.randn()
        y = gS.bulletClient.getJointState(
            gS.gantryId, gS.GantryLinkIndex_dict["BasePositionIndex"])[0] + 0.00 * np.random.randn()
        z = gS.bulletClient.getJointState(
            gS.gantryId, gS.GantryLinkIndex_dict["ZAxisBarIndex"])[0] + 0.00 * np.random.randn()
        # JawRadialPos = gS.bulletClient.getJointState(
        #     gS.gantryId, gS.gantryLinkDict["SJ1"])[0]
        force_feedback_1 = gS.bulletClient.getContactPoints(
            gS.gantryId, gS.objects["PickupCube"].objId, gS.gantryLinkDict["SJ1"], -1)
        force_feedback_2 = gS.bulletClient.getContactPoints(
            gS.gantryId, gS.objects["PickupCube"].objId, gS.gantryLinkDict["SJ2"], -1)
        force_feedback_3 = gS.bulletClient.getContactPoints(
            gS.gantryId, gS.objects["PickupCube"].objId, gS.gantryLinkDict["SJ3"], -1)
        if len(force_feedback_1) != 0:
            force_1 = np.linalg.norm(sum(np.array([np.array(x[7])*x[9] for x in force_feedback_1])),2)
        else:
            force_1 = 0
        if len(force_feedback_2) != 0:
            force_2 = np.linalg.norm(sum(np.array([np.array(x[7])*x[9] for x in force_feedback_2])),2)
        else:
            force_2 = 0
        if len(force_feedback_3) != 0:
            force_3 = np.linalg.norm(sum(np.array([np.array(x[7])*x[9] for x in force_feedback_3])),2)
        else:
            force_3 = 0
        gripper_position = torch.Tensor([x, y, z]).unsqueeze(dim=0)
        force = torch.Tensor([force_1, force_2, force_3]).unsqueeze(dim=0)
        #print("force_1:",str(force_1))
        #print("force_2:",str(force_2))
        #print("force_3:",str(force_3))

        if GUI_control is False:
            commands = perceptor.forward(
                gripper_position, object_position, target_position, force)
            [move_to_pre_grasp, move_to_grasp, grasp, lift_after_grasp, move_to_pre_release,
                move_to_release, release, lift_after_release] = commands.squeeze(dim=0).numpy()
            [x_d, y_d, z_d, JawRadialPos] = controller.forward(
                object_position, target_position, commands).numpy()
            print("JawPos:",str(JawRadialPos))
            if lift_after_release > 10:
                object_position = torch.Tensor([0, 0, 0]).unsqueeze(dim=0)

            positionset.append([x,y,z])
            targetpositionset.append([x_d,y_d,z_d])
            forceset.append([force_1, force_2, force_3])
            neuronset.append([move_to_pre_grasp, move_to_grasp, grasp, lift_after_grasp, move_to_pre_release, move_to_release, release, lift_after_release])

        else:
            [x_d, y_d, z_d, JawRadialPos] = [0, 0, 0, 0]



        GrasperArguments = {"frictionCoefficient":1,"PressureValue":2.5, #change the pressure value to see change in effective stiffness.
                             "TargetJawPosition":JawRadialPos, "MaxJawForce":20, "MaxVel":0.1,
                             "MaxVertForce":100,
                             "TargetVertPosition":0, "MaxVertVel":0.1}

        ArgumentDict = {"x_gantryHead": x_d, "y_BasePos": y_d, "z_AxisBar": z_d, "x_force": 50, "y_force": 500,
                        "z_force": 500, "GrasperArguments": GrasperArguments}

        # ---------step the simulation----------
        gS.stepSim(usePositionControl=True, GUI_override=True, **ArgumentDict)  # pass argument dict to function

    return positionset, targetpositionset, forceset, neuronset

In [3]:
positionset, targetpositionset, forceset, neuronset, sensoryset = pick_and_place()

Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
Programmatic Control
BasePlateJoint:[ 2.33178570e-12  3.62191113e-04 -9.86035633e+01]
Programmatic Control
BasePlateJoint:[ 9.00527520e-13  2.64060694e-04 -7.18883609e+01]
Programmatic Control
BasePlateJoint:[ 8.85798212e-14  2.60519693e-05 -7.09243523e+00]
Programmatic Control
BasePlateJoint:[ 8.92463713e-14  2.65557980e-05 -7.22959848e+00]
Programmatic Control
BasePlateJoint:[ 4.04009888e-14  2.69549682e-05 -7.33826931e+00]
Programmatic Control
BasePlateJoint:[ 1.75978183e-13  2.71127138e-05 -7.38121425e+00]
Programmatic Control
BasePlateJoint:[ 1.08072403e-13  2.71817292e-05 -7.40000315e+00]
Programmatic Control
BasePlateJoint:[-1.01431262e-14  2.70982768e-05 -7.37728394e+0

ValueError: not enough values to unpack (expected 5, got 4)

In [ ]:
position_set = np.array(positionset)
target_position_set = np.array(targetpositionset)
force_set = np.array(forceset)
neuron_set = np.array(neuronset)
neuron_set[neuron_set<0] = 0
sensory_set = np.array(sensoryset)

plt.figure()
plt.subplot(211)
plt.plot(np.arange(len(target_position_set[:,-1]))/240,neuron_set[:,7])
plt.ylabel('Noisy feedback (x)')
#plt.plot(target_position_set[:,-1])

plt.subplot(212)
#plt.plot(np.argmax(neuron_set,axis=1))
plt.plot(np.arange(len(target_position_set[:,-1]))/240,neuron_set[:,7])
plt.plot(np.arange(len(target_position_set[:,-1]))/240,sensory_set[:,3])
plt.xlabel('Time (s)')
plt.ylabel('move_to_pre_grasp')
plt.show()